In [20]:
import os

MODEL_PATH = r"C:\content\drive\MyDrive\FYP\models\20250415_1810\classifier.pkl"

if not os.path.exists(MODEL_PATH):
    print(f"Error: The model file does not exist at {MODEL_PATH}")
else:
    print("File found!")

File found!


In [22]:
file_size = os.path.getsize(MODEL_PATH)
print(f"File size: {file_size} bytes")

File size: 0 bytes


In [35]:
with open(MODEL_PATH, "rb") as f:
    content = f.read(100)  # Read the first 100 bytes
    print(content)

b'\x80\x04\x95\xd2\x02\x00\x00\x00\x00\x00\x00\x8c\x14sklearn.ensemble._gb\x94\x8c\x1aGradientBoostingClassifier\x94\x93\x94)\x81\x94}\x94(\x8c\x0cn_estimators\x94Kd\x8c\rlearning_r'


In [4]:


%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
import hashlib
import json
import os
from datetime import datetime
import requests
from sklearn.ensemble import GradientBoostingClassifier
import joblib

# - Configuration -
MODEL_PATH = r"C:\Users\Forge-15 i7\models\20250415_190129\classifier.pkl"
METADATA_PATH = r"C:\Users\Forge-15 i7\models\20250415_190129\metadata.json"
SENSOR_DATA_FILE = "sensor_data.json"  # Local file to store sensor data temporarily

# Load the pre-trained ML model
try:
    model = joblib.load(MODEL_PATH)
    print("Model loaded successfully!")
except Exception as e:
    st.error(f"Error loading model: {e}")
    st.stop()

# Load metadata (expected features)
try:
    with open(METADATA_PATH, "r") as f:
        metadata = json.load(f)
        FEATURES = metadata["features"]
        print("Expected features:", FEATURES)
except Exception as e:
    st.error(f"Error loading metadata: {e}")
    st.stop()

# Set page configuration
st.set_page_config(
    page_title="Food Spoilage Monitoring",
    page_icon="🍏",
    layout="wide",
    initial_sidebar_state="expanded"
)

st.markdown("""
<style>
    /* Change background color */
    .stApp {
        background-color: #000000;
    }

    /* Style headers */
    h1 {
        color: #2e7d32; /* Dark green */
        font-family: 'Arial', sans-serif;
    }

    /* Style buttons */
    .stButton > button {
        background-color: #4caf50; /* Green */
        border: none;
        color: white;
        padding: 10px 20px;
        text-align: center;
        text-decoration: none;
        display: inline-block;
        font-size: 16px;
        margin: 4px 2px;
        cursor: pointer;
        border-radius: 8px;
    }

    /* Hover effect for buttons */
    .stButton > button:hover {
        background-color: #45a049; /* Slightly darker green */
    }

    /* Style dataframes */
    .stDataFrame {
        background-color: #062900;
        border-radius: 8px;
        box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
    }

    /* Sidebar styling */
    .css-1d391kg {
        background-color: #062900; /* Light green */
        border-radius: 10px;
    }
</style>
""", unsafe_allow_html=True)

# Initialize session state for login
if "logged_in" not in st.session_state:
    st.session_state["logged_in"] = False
    st.session_state["user"] = None

# Function to hash passwords
def hash_password(password):
    return hashlib.sha256(password.encode()).hexdigest()

# Load users from a local file (simulated database)
def load_users():
    if os.path.exists("users.json"):
        with open("users.json", "r") as f:
            return json.load(f)
    return {}

# Save users to a local file
def save_users(users):
    with open("users.json", "w") as f:
        json.dump(users, f)

# Sign-up function
def sign_up():
    st.subheader("📝 Sign Up")
    new_user = st.text_input("Username", key="signup_username")
    new_password = st.text_input("Password", type="password", key="signup_password")
    confirm_password = st.text_input("Confirm Password", type="password", key="confirm_password")

    if st.button("Sign Up"):
        users = load_users()
        if new_user in users:
            st.error("❌ Username already exists!")
        elif new_password != confirm_password:
            st.error("❌ Passwords do not match!")
        else:
            users[new_user] = hash_password(new_password)
            save_users(users)
            st.success("✅ Account created! Please log in.")

# Login function
def login():
    st.subheader("🔑 Login to Dashboard")
    username = st.text_input("Username", key="login_username")
    password = st.text_input("Password", type="password", key="login_password")

    if st.button("Login"):
        users = load_users()
        if username in users and users[username] == hash_password(password):
            st.session_state["logged_in"] = True
            st.session_state["user"] = username
            st.rerun()  # Refresh the app to update the UI
        else:
            st.error("❌ Invalid username or password!")

# Fetch sensor data (simulated)
def fetch_sensor_data():
    if os.path.exists(SENSOR_DATA_FILE):
        with open(SENSOR_DATA_FILE, "r") as file:
            data_list = json.load(file)
        return pd.DataFrame(data_list[-1:])  # Return only the latest reading
    return pd.DataFrame()

def preprocess_data(df):
    """
    Preprocesses the raw data fetched from Flask to match the format expected by the ML model.
    """
    # Ensure required base columns exist
    required_base_columns = ['MQ135', 'MQ3', 'MQ4', 'temperature', 'humidity', 'timestamp']
    
    # If column names are mismatched, rename them
    column_mapping = {
        "MQ-135": "MQ135",
        "MQ-3": "MQ3",
        "MQ-4": "MQ4",
        "Temperature": "temperature",
        "Humidity": "humidity",
        "device_ID": "device_id",  # Handle potential mismatches
        "deviceId": "device_id"
    }
    df.rename(columns=column_mapping, inplace=True)

    # Add default device_id if missing
    if 'device_id' not in df.columns:
        df['device_id'] = 'default_device_id'

    # Recheck for missing base columns after renaming
    missing_base_columns = [col for col in required_base_columns if col not in df.columns]
    if missing_base_columns:
        raise KeyError(f"Missing base columns: {missing_base_columns}")

    # Convert timestamp to datetime
    try:
        df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')  # Handle invalid timestamps
    except Exception as e:
        raise ValueError(f"Error converting timestamp: {e}")

    # Create time features
    df['hour'] = df['timestamp'].dt.hour
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)

    # Create rolling means for sensors
    sensors = ['MQ135', 'MQ3', 'MQ4', 'temperature', 'humidity']
    for sensor in sensors:
        try:
            df[f'{sensor}_3h_mean'] = (
                df.groupby('device_id')[sensor]
                .rolling(window=3, min_periods=1)
                .mean()
                .reset_index(level=0, drop=True)
            )
        except Exception as e:
            raise RuntimeError(f"Error creating rolling mean for {sensor}: {e}")

    # Add MQ_ratio_135_3 if not already present
    if 'MQ_ratio_135_3' not in df.columns:
        try:
            df['MQ_ratio_135_3'] = df['MQ135_3h_mean'] / (df['MQ3_3h_mean'] + 1e-6)  # Avoid division by zero
        except Exception as e:
            raise RuntimeError(f"Error creating MQ_ratio_135_3: {e}")

    # Select features used by the ML model
    required_features = [
        'MQ135_3h_mean', 'MQ3_3h_mean', 'MQ4_3h_mean',
        'temperature_3h_mean', 'humidity_3h_mean',
        'hour_sin', 'hour_cos', 'MQ_ratio_135_3'
    ]

    # Validate feature creation
    missing_features = [f for f in required_features if f not in df.columns]
    if missing_features:
        raise KeyError(f"Missing features: {missing_features}")

    return df[required_features]

# Display sensor readings as bar charts
def display_sensor_readings(sensor_data):
    st.header("📊 Real-Time Sensor Readings")
    st.markdown("Below are the latest sensor readings displayed as bar charts.")

    # Define colors for each sensor
    colors = {
        "MQ-135": "#1f77b4",  # Blue
        "MQ-3": "#ff7f0e",   # Orange
        "MQ-4": "#2ca02c",   # Green
        "Humidity": "#d62728",  # Red
        "Temperature": "#9467bd"  # Purple
    }

    # Create columns for the bar charts
    col1, col2, col3, col4, col5 = st.columns(5)

    # List of sensors
    sensors = ["MQ-135", "MQ-3", "MQ-4", "Humidity", "Temperature"]

    # Plot bar charts for each sensor
    for i, sensor in enumerate(sensors):
        fig = px.bar(
            x=["Current Reading"],  # X-axis label
            y=[sensor_data[sensor].iloc[0]],  # Y-axis value
            title=f"{sensor} ({sensor_data[sensor].iloc[0]:.1f})",
            color_discrete_sequence=[colors[sensor]]  # Set custom color
        )
        fig.update_layout(showlegend=False)  # Hide legend

        # Display the chart in the appropriate column
        if i == 0:
            col1.plotly_chart(fig, use_container_width=True)
        elif i == 1:
            col2.plotly_chart(fig, use_container_width=True)
        elif i == 2:
            col3.plotly_chart(fig, use_container_width=True)
        elif i == 3:
            col4.plotly_chart(fig, use_container_width=True)
        elif i == 4:
            col5.plotly_chart(fig, use_container_width=True)
            
# Add a new page for Gas Emission Trends
def display_gas_emission_trends():
    st.header("📈 Gas Emission Trends")
    st.markdown("Analyze trends in gas emissions over time.")

    # Load historical data
    if os.path.exists(SENSOR_DATA_FILE):
        with open(SENSOR_DATA_FILE, "r") as file:
            data_list = json.load(file)
            historical_data = pd.DataFrame(data_list)
    else:
        historical_data = pd.DataFrame()

    if historical_data.empty:
        st.warning("⚠️ Not enough data for gas emission trends yet.")
        return

    # Ensure required columns exist
    required_columns = ["timestamp", "MQ-135", "MQ-3", "MQ-4"]
    missing_columns = [col for col in required_columns if col not in historical_data.columns]
    if missing_columns:
        st.error(f"Error: Missing required columns for gas emission trends: {missing_columns}")
        return

    # Convert timestamp to datetime
    historical_data["timestamp"] = pd.to_datetime(historical_data["timestamp"])

    # Line chart for gas emission trends
    fig = px.line(
        historical_data,
        x="timestamp",
        y=["MQ-135", "MQ-3", "MQ-4"],
        title="Gas Emission Trends Over Time",
        labels={"value": "Gas Concentration", "timestamp": "Time"},
        color_discrete_map={
            "MQ-135": "#1f77b4",  # Blue
            "MQ-3": "#ff7f0e",   # Orange
            "MQ-4": "#2ca02c"    # Green
        }
    )
    fig.update_layout(
        legend_title="Gas Sensors",
        hovermode="x unified"
    )

    st.plotly_chart(fig, use_container_width=True)

# Display predictions as doughnut charts
def display_predictions(predictions):
    st.header("🤖 Predictions")
    st.markdown("Below are the predictions displayed as doughnut charts.")

    spoilage_probability = predictions["probability"] * 100
    freshness_probability = 100 - spoilage_probability

    fig = px.pie(
        names=["Spoiled", "Fresh"],
        values=[spoilage_probability, freshness_probability],
        hole=0.5,
        title=f"Spoilage Probability: {spoilage_probability:.1f}%"
    )
    fig.update_traces(marker=dict(colors=["#d62728", "#2ca02c"]))  # Red for spoiled, green for fresh
    fig.update_layout(showlegend=False)

    st.plotly_chart(fig, use_container_width=True)

# Dashboard function
def dashboard():
    st.title("🍽️ Food Spoilage Monitoring Dashboard")
    st.markdown(f"Welcome, **{st.session_state['user']}**! Below are the real-time sensor readings and analytics.")

    # Sidebar for navigation
    st.sidebar.title("Navigation")
    page = st.sidebar.radio(
        "Go to",
        ["Sensor Readings", "Predictions","Gas Emission Trends"]
    )
    st.sidebar.button("Logout", on_click=lambda: logout())

    # Fetch sensor data
    sensor_data = fetch_sensor_data()
    if sensor_data.empty:
        st.warning("⚠️ No sensor data available yet.")
        return

    # Render the selected page
    if page == "Sensor Readings":
        display_sensor_readings(sensor_data)
    elif page == "Predictions":
        # Preprocess data for prediction
        processed_data = preprocess_data(sensor_data)
        # Validate feature alignment
        missing_features = [f for f in FEATURES if f not in processed_data.columns]
        if missing_features:
            st.error(f"Error: Missing features in processed data: {missing_features}")
            return
        try:
            predictions = model.predict(processed_data)
            probability = model.predict_proba(processed_data)[0][1]
            result = {
                "prediction": "Spoiled" if probability > 0.5 else "Fresh",
                "probability": probability
            }
            display_predictions(result)
        except Exception as e:
            st.error(f"Error during prediction: {e}")
    elif page == "Gas Emission Trends":
        display_gas_emission_trends()
        
# Logout function
def logout():
    st.session_state["logged_in"] = False
    st.session_state["user"] = None
    st.success("Logged out successfully!")
    st.rerun()

# Main function
if __name__ == "__main__":
    if not st.session_state["logged_in"]:
        menu = st.sidebar.selectbox("Menu", ["Login", "Sign Up"])
        if menu == "Login":
            login()
        else:
            sign_up()
    else:
        dashboard()

Overwriting app.py
